In [1]:
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import tqdm
from SenseTheFlow import config
config.bar = tqdm.tqdm_notebook

from SenseTheFlow.model import Model, CustomSummarySaverHook, DataParser, EvalCallback
from SenseTheFlow import rocksdb
from SenseTheFlow.dataset import Dataset, CustomLoader

from util import *

LMDB Loading won't be available


In [3]:
import mnist
import random
from scipy.ndimage.interpolation import rotate

def train_generator():
    images, labels = mnist.train_images(), mnist.train_labels()
    for i in range(images.shape[0]):
        img, label = images[i, :, :, np.newaxis], labels[i]
        img = img.astype(np.float32)
        cls = random.randint(0, 1)
        
        if cls == 1:
            img = rotate(img, 15, reshape=False)
            
        yield (img, {'label': [int(label)], 'cls': [int(cls)]})

In [4]:
def crossgrad_latent_fn(features, labels, mode, params):
    # Some convolutions
    features = tf.layers.conv2d(features, 32, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.conv2d(features, 32, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.max_pooling2d(features, 2, 1, data_format=params['data_format'])
    features = tf.layers.conv2d(features, 128, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.conv2d(features, 128, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.max_pooling2d(features, 2, 1, data_format=params['data_format'])
    features = tf.layers.conv2d(features, 256, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.conv2d(features, 256, 3, data_format=params['data_format'], activation=tf.nn.relu)
    
    # Down to [batch, 1, 1, 256] -> [batch, 1, 1, 256]
    size = features.shape.as_list()[2]
    features = tf.layers.max_pooling2d(features, size, 1, data_format=params['data_format'])

    # Target dimensions [batch, 1, 1, target] -> [batch, target]
    features = tf.layers.conv2d(features, params['latent_space_dimensions'], 1, data_format=params['data_format'])
    features = tf.layers.flatten(features)
    
    return features

In [5]:
def crossgrad_domain_fn(features, labels, mode, params):
    # Compute logits
    features = tf.layers.dense(features, params['latent_space_dimensions'], activation=tf.nn.relu)
    logits = tf.layers.dense(features, params['num_domain'])
    
    # Final classification
    softmax = tf.nn.softmax(logits)
    domain = tf.argmax(softmax, axis=-1)
    
    # Loss
    loss = tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.one_hot(labels, params['num_domain']),
        logits=logits
    )
    
    return domain, tf.reduce_mean(loss)

In [6]:
def crossgrad(mode, label_fn, x, domain, labels, epsilon_d, epsilon_l, alpha_d, alpha_l, latent_fn=None, domain_fn=None, latent_space_dimensions=100, params=None):    
    # Some default parameters
    if params == None: params = {}        
    params['latent_space_dimensions'] = latent_space_dimensions
    if 'data_format' not in params or params['data_format'] is None:
        params['data_format'] = detect_data_format()
    
    # Given functions or default
    latent_fn = latent_fn or crossgrad_latent_fn
    domain_fn = domain_fn or crossgrad_domain_fn
    
    # To expected data format
    x = to_data_format(x, 'channels_last', params['data_format'])
    
    def forward(x_l, x_d, d, labels, mode, params):
        latent = latent_fn(features=x_l, labels=None, mode=mode, params=params)
        domain, loss_domain = domain_fn(features=latent, labels=d, mode=mode, params=params)
        outputs, loss_label = label_fn(features=x_d, latent=latent, labels=labels, mode=mode, params=params)

        return ((domain, loss_domain), (outputs, loss_label))
    
    # First forward pass
    ((d1, d1_loss), (l1, l1_loss)) = forward(x, x, domain, labels, mode, params)
    grad_label_x = tf.gradients(l1_loss, x)[0]
    grad_domain_x = tf.gradients(d1_loss, x)[0]
    
    tf.summary.scalar('loss/domain/1', d1_loss)
    tf.summary.scalar('loss/labels/1', l1_loss)
    
    # Second pass
    x_d = x + epsilon_d * grad_domain_x
    x_l = x + epsilon_l * grad_label_x
    ((d2, d2_loss), (l2, l2_loss)) = forward(x_l, x_d, domain, labels, mode, params)
    
    tf.summary.scalar('loss/domain/2', d2_loss)
    tf.summary.scalar('loss/labels/2', l2_loss)    
    
    tf.summary.image('x', to_data_format(x, params['data_format'], 'channels_last'))
    tf.summary.image('x_d', to_data_format(x_d, params['data_format'], 'channels_last'))
    tf.summary.image('x_l', to_data_format(x_l, params['data_format'], 'channels_last'))
    
    l_total_loss = (1 - alpha_l) * l1_loss + alpha_l * l2_loss
    d_total_loss = (1 - alpha_d) * d1_loss + alpha_d * d2_loss
    predictions = {
        'd1': d1,
        'd2': d2,
        'l1': l1,
        'l2': l2
    }
    return predictions, l_total_loss + d_total_loss

In [7]:
def label_fn(features, latent, labels, mode, params):
    features = tf.layers.conv2d(features, 32, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.conv2d(features, 64, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.max_pooling2d(features, 2, 1, data_format=params['data_format'])
    features = tf.layers.flatten(features)
    features = tf.layers.dense(features, params['latent_space_dimensions'], activation=tf.nn.relu)
    features = tf.concat((features, latent), axis=-1)
    features = tf.layers.dense(features, params['latent_space_dimensions'], activation=tf.nn.relu)
    logits = tf.layers.dense(features, params['num_classes'])
    
    print('LOGITS_LABELS')
    print(logits)
    
    # Final classification
    softmax = tf.nn.softmax(logits)
    cls = tf.argmax(softmax, axis=-1)
    
    # Loss
    loss = tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.one_hot(labels, params['num_classes']),
        logits=logits
    )
    
    return cls, tf.reduce_mean(loss)

In [ ]:
def model_fn(features, labels, mode, params):
    print(features, labels)
    with tf.device('/gpu:1'):
        predictions, loss = crossgrad(mode, label_fn, features, labels['cls'], labels['label'], 2.0, 2.0, 0.2, 0.2, params=params)
        
        print('done')
        print(predictions['d1'])
        print(labels['cls'])
        print(predictions['d1'])
        print(labels['cls'])
        
        accuracy_d1 = tf.metrics.accuracy(predictions['d1'], labels['cls'])
        accuracy_l1 = tf.metrics.accuracy(predictions['l1'], labels['label'])
        tf.summary.scalar('metrics/epoch/accuracy/domain/1', accuracy_d1[1])
        tf.summary.scalar('metrics/epoch/accuracy/labels/1', accuracy_l1[1])
        
        accuracy_d2 = tf.metrics.accuracy(predictions['d2'], labels['cls'])
        accuracy_l2 = tf.metrics.accuracy(predictions['l2'], labels['label'])
        tf.summary.scalar('metrics/epoch/accuracy/domain/2', accuracy_d2[1])
        tf.summary.scalar('metrics/epoch/accuracy/labels/2', accuracy_l2[1])
        
        
        # Fetch global step
        global_step = tf.train.get_or_create_global_step()
        tf.summary.scalar('global_step/step', global_step)

        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

            # Batch norm requires update_ops to be added as a train_op dependency.
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops + [accuracy_d1[0], accuracy_l1[0], accuracy_d2[0], accuracy_l2[0]]):
                train_op = optimizer.minimize(loss, global_step)
        else:
            train_op = None

        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            loss=loss,
            train_op=train_op,
            eval_metric_ops={
                'accuracy/domain/1': accuracy_d1, 
                'accuracy/labels/1': accuracy_l1,
                'accuracy/domain/2': accuracy_d2, 
                'accuracy/labels/2': accuracy_l2
            })

In [ ]:
params = {
    'data_format': None,
    'batch_size': 32,
    'num_domain': 2,
    'num_classes': 10
}

model_dir = '/data/storage/deepglobe/models/{}'.format('crossgrad')

config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
config.gpu_options.allow_growth = True

tf.logging.set_verbosity(tf.logging.ERROR)

with Model(model_fn, model_dir, config=config, params=params, delete_existing=True) as model:
    data_parser = DataParser()
    data_parser.train_from_generator(
        generator=train_generator,
        output_types=(tf.float32, {'label': tf.int32, 'cls': tf.int32}),
        output_shapes=([28, 28, 1], {'label': [1], 'cls': [1]}),
        pre_shuffle=7,
        post_shuffle=False,
        flatten=False,
        num_samples=None,
        batch_size=params['batch_size']
    )
     
    model.data(data_parser)
    
    model.train(100, 2, eval_summary=[
        'metrics/batch/accuracy', 'metrics/epoch/accuracy', 'metrics/epoch/miou'
    ])

#     do_predict(model)
#     do_train_pred(model)
#     do_eval_pred(model)

Tensor("IteratorGetNext:0", shape=(?, 28, 28, 1), dtype=float32, device=/device:CPU:0) {'label': <tf.Tensor 'IteratorGetNext:2' shape=(?, 1) dtype=int32>, 'cls': <tf.Tensor 'IteratorGetNext:1' shape=(?, 1) dtype=int32>}
LOGITS_LABELS
Tensor("dense_4/BiasAdd:0", shape=(?, 10), dtype=float32, device=/device:GPU:1)
LOGITS_LABELS
Tensor("dense_9/BiasAdd:0", shape=(?, 10), dtype=float32, device=/device:GPU:1)
done
Tensor("ArgMax:0", shape=(?,), dtype=int64, device=/device:GPU:1)
Tensor("IteratorGetNext:1", shape=(?, 1), dtype=int32, device=/device:CPU:0)
Tensor("ArgMax:0", shape=(?,), dtype=int64, device=/device:GPU:1)
Tensor("IteratorGetNext:1", shape=(?, 1), dtype=int32, device=/device:CPU:0)
You have no `evaluation` dataset
Tensor("IteratorGetNext:0", shape=(?, 28, 28, 1), dtype=float32, device=/device:CPU:0) {'label': <tf.Tensor 'IteratorGetNext:2' shape=(?, 1) dtype=int32>, 'cls': <tf.Tensor 'IteratorGetNext:1' shape=(?, 1) dtype=int32>}
LOGITS_LABELS
Tensor("dense_4/BiasAdd:0", shape=